In [1]:
val_csv_path = '/kaggle/input/iiit-hyd-smai-project-dataset/labels_val.csv'
train_csv_path = '/kaggle/input/iiit-hyd-smai-project-dataset/labels_train.csv'
train_img_dir = '/kaggle/input/iiit-hyd-smai-project-dataset/images_train'
val_img_dir = '/kaggle/input/iiit-hyd-smai-project-dataset/images_val'
test_img_dir = '/kaggle/input/iiit-hyd-smai-project-dataset/images_test'

# Output files will be written to the working directory
lat_long_output_csv = '/kaggle/working/lat-long.csv'
region_output_csv = '/kaggle/working/region.csv'
angle_output_csv = '/kaggle/working/angle.csv'
print(val_csv_path)

/kaggle/input/iiit-hyd-smai-project-dataset/labels_val.csv


In [2]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, models
from torchvision.models import resnet50, ResNet50_Weights
model = resnet50(weights=ResNet50_Weights.DEFAULT)
import timm

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 186MB/s]


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"💻 Using device: {device}")

💻 Using device: cuda


In [4]:
train_df = pd.read_csv(train_csv_path)
val_df = pd.read_csv(val_csv_path)
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = val_transform

In [5]:
class RegionDataset(Dataset):
    def __init__(self, img_dir, df=None, transform=None, is_test=False):
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test

        if not is_test:
            self.df = df.reset_index(drop=True)
            self.filenames = self.df['filename'].tolist()
            print(f"Initialized dataset with {len(self.df)} samples from {img_dir}")
        else:
            self.filenames = sorted([
                f for f in os.listdir(img_dir)
                if f.lower().endswith(('.jpg', '.jpeg', '.png'))
            ])
            print(f"Initialized test dataset with {len(self.filenames)} images from {img_dir}")

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        filename = self.filenames[idx]
        img_path = os.path.join(self.img_dir, filename)
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:
            return image, filename  # No label
        else:
            label = self.df.iloc[idx]['Region_ID'] - 1
            return image, label, filename

train_dataset = RegionDataset(train_img_dir, df=train_df, transform=train_transform)
val_dataset = RegionDataset(val_img_dir, df=val_df, transform=val_transform)

test_dataset = RegionDataset(test_img_dir, transform=test_transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=64, num_workers=2, pin_memory=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, num_workers=2, pin_memory=True)

Initialized dataset with 6542 samples from /kaggle/input/iiit-hyd-smai-project-dataset/images_train
Initialized dataset with 369 samples from /kaggle/input/iiit-hyd-smai-project-dataset/images_val
Initialized test dataset with 369 images from /kaggle/input/iiit-hyd-smai-project-dataset/images_test


In [6]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True
model.fc = nn.Linear(model.fc.in_features, 15)
model = model.to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

best_val_acc = 0
best_epoch = 0
patience = 4
region_vectors = {}

print("🚦 Starting training with early stopping...")
for epoch in range(20):  # Max 20 epochs
    model.train()
    running_loss = 0
    pbar = tqdm(train_loader, desc=f"📚 Epoch {epoch+1}/20", leave=False)
    for images, labels, filenames in pbar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        pbar.set_postfix({'Loss': f"{running_loss / (pbar.n + 1):.4f}"})

        # Collect region IDs for train samples
        for fname, label in zip(filenames, labels.cpu().numpy()):
            region_vectors[fname] = label + 1  # back to 1-indexed

    avg_loss = running_loss / len(train_loader)

    # Validation
    model.eval()
    predictions, ground_truth = [], []
    with torch.no_grad():
        for images, labels, _ in val_loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            ground_truth.extend(labels.numpy())

    val_acc = accuracy_score(ground_truth, predictions)
    print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | Validation Accuracy: {val_acc:.4f}")
    scheduler.step()

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch
        best_model_state = model.state_dict()

    # Early stopping
    if epoch - best_epoch >= patience:
        print(f"⏹ Early stopping at epoch {epoch+1}")
        break

if best_model_state is not None:
    torch.save(best_model_state, "/kaggle/working/best_region_model.pth")
    print(f"Saved best model from epoch {best_epoch+1} with val acc: {best_val_acc:.4f}")

🚦 Starting training with early stopping...


Epoch 1 | Loss: 2.6167 | Validation Accuracy: 0.1653


Epoch 2 | Loss: 2.4867 | Validation Accuracy: 0.2385


Epoch 3 | Loss: 2.3879 | Validation Accuracy: 0.2818


Epoch 4 | Loss: 2.3017 | Validation Accuracy: 0.3523


...

Epoch 16 | Loss: 1.9573 | Validation Accuracy: 0.4607


Epoch 17 | Loss: 1.9506 | Validation Accuracy: 0.4878


Epoch 18 | Loss: 1.9518 | Validation Accuracy: 0.5014


Epoch 19 | Loss: 1.9468 | Validation Accuracy: 0.4770


Epoch 20 | Loss: 1.9406 | Validation Accuracy: 0.4688
Saved best model from epoch 18 with val acc: 0.5014


In [7]:
model.eval()
predictions = []
with torch.no_grad():
    for images, _, _ in val_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())

val_preds_df = pd.DataFrame({
    'id': list(range(369)),
    'Region_ID': [p + 1 for p in predictions]
})

model.eval()

test_predictions = []
test_filenames = []

with torch.no_grad():
    for images, filenames in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        test_predictions.extend(preds.cpu().numpy())
        test_filenames.extend(filenames)

test_preds_df = pd.DataFrame({
    'id': list(range(369, 738)),
    'Region_ID': [p + 1 for p in test_predictions]
})

submission_df = pd.concat([val_preds_df, test_preds_df], ignore_index=True)
submission_df.to_csv(region_output_csv, index=False)
print(f"Region submission saved to {region_output_csv}")

Region submission saved to /kaggle/working/region.csv
